# Recycle systems

## Example

![BuOH_separation](../images/BuOH_separation.png)

Recycle systems in BioSTEAM are solved through accelerated fixed-point iteration (just like in Aspen Plus and SuperPro). Let's create an industry-relevant system for the purification of butanol from fermentation broth in 2 steps:

1. Create and solve the network of unit operations. 

2. Visualize the recycle system.


**Step 1: Build the system**

In [ ]:
import biosteam as bst
bst.nbtutorial()
bst.settings.set_thermo(['Water', 'Butanol'], cache=True)
with bst.System() as sys:
    feed = bst.Stream(
        'feed', 
        phase='l', 
        Water=900,
        Butanol=100,
        units='kmol/hr',
    )
    water_rich = bst.Stream('water_rich')
    distillate_0 = bst.Stream('distillate_0')
    distillate_1 = bst.Stream('distillate_1')
    
    # Dewater
    water_distiller = bst.BinaryDistillation(
        ins=(feed, water_rich), outs=(distillate_0, 'water'),
        x_bot=0.0001, y_top=0.2, k=1.2, Rmin=0.01,
        LHK=('Butanol', 'Water'),
        partial_condenser=False,
    )
    
    # Decanter
    mixer = bst.Mixer('mixer', ins=(distillate_0, distillate_1))
    settler = bst.StageEquilibrium(
        'settler',
        ins=mixer-0, 
        outs=('butanol_rich', water_rich),
        phases=('L', 'l'),
        top_chemical='Butanol',
        T=310,
    )
    
    # Butanol purification
    butanol_distiller = bst.BinaryDistillation(
        ins=settler-0,
        outs=(distillate_1, 'butanol'),
        x_bot=0.0001, y_top=0.6, 
        k=1.2, Rmin=0.01,
        LHK=('Water', 'Butanol'),
        partial_condenser=False,
    )
sys.simulate()
sys.show()

**Step 2: Visualize the system**

Pass `number=True` to label the simulation order in the diagram:

In [ ]:
sys.diagram(number=True, format='svg')

This topography shows a flat system without explicit recycle loops. To view the nested system, pass `kind='cluster'`:

In [ ]:
sys.diagram(number=True, kind='cluster', format='svg')

Some unit operations are composed of multiple unit operations that are solved internally with specialized algorithms. To view the first layer of these auxiliary units, pass `auxiliaries=1`:

In [ ]:
sys.diagram(auxiliaries=1, format='svg')

## Problem

As an excericise, let's look model a common configuration for distillation with heat integration. We will create and analyse a beer column with heat integration for the separation of ethanol from fermentation broth. 

**Instructions:** Replace all `None`s with the correct values.

In [ ]:
import biosteam as bst
bst.nbtutorial()
bst.settings.set_thermo(['Water', 'Ethanol'])
with bst.System() as sys:
    feed = bst.Stream(
        'feed',
        Ethanol=12,
        Water=88,
        total_flow=1.5e5,
        units='kg/hr',
    )
    pump = bst.Pump(ins=None, P=6 * 101325)
    recycle = bst.Stream('recycle')
    heat_exchanger = bst.HXprocess(
        ins=(None, None), 
        outs=('hot_feed', 'cooled_bottoms_product'),
    )
    beer_column = bst.BinaryDistillation(# ShortcutColumn for 3+ components
        ins=None,
        outs=('distillate', 'bottoms_product'), 
        LHK=('Ethanol', 'Water'), # Light and heavy key
        P=2.1 * 101325, # Operating pressure [Pa]
        y_top=0.35, # Distillate light-key fraction
        x_bot=1e-5, # Bottoms heavy-key fraction
        k=1.1, # Actual reflux over minimum reflux ratio
    )
    pump = bst.Pump(
        ins=None, 
        outs=None, # Outlet goes to heat exchanger
        P=4 * 101325
    )
sys.simulate()
sys.show()

In [ ]:
sys.diagram(number=True, kind='cluster', format='svg')